# 04 Wind factor calculations with ERA5 data and OSI_SAF data

Code adapted from previous codes

### Import packages

In [1]:
# general
import numpy as np, numpy.ma as ma
import xarray as xr
import pandas as pd

# time
from datetime import datetime, timedelta
import matplotlib.dates as mdates

# local system 
import sys  
import glob
import os

# plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.colors
import cmocean

# geo plotting
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeat

# for use in suppressing repeated warnings when mapping w/ shapely
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

# path to own functions
sys.path.append('../Libraries_functions/')
from LIB_ASI_SIC_UniB import grab_ASI_SIC, grab_projinfo_SIC
from LIB_geo_func import *
from LIB_geo_plot import *


# OSI SAF sea ice drift
from LIB_OSI_SAF import grab_projinfo_OSISAF, grab_OSISAF_drift

# NSIDC sea ice drift
from LIB_PPdrift_NSIDC0116 import grab_projinfo_PPdrift


%load_ext autoreload
%autoreload 2
# potentially uninstall pyhdf?

#math
import math

### Sea ice drift 

In [ ]:

# specify date to import
#============================
date = datetime(2024, 1, 14)
#============================

OSISAF = grab_OSISAF_drift(date = date, 
                          main_path = '/Volumes/Seagate2/OSI_SAF_Drift/')


# OSISAF sea ice drift for mapping
qv1 = ax_map.quiver(OSISAF['x'], OSISAF['y'], OSISAF['dx']/2, OSISAF['dy']/2, 
               color = 'gray', width = 0.002, headwidth=5, scale = 500, transform = OSISAF['proj'], zorder=5)
qk = ax_map.quiverkey(qv1, 0.9, 0.1, 20, r'$20 \frac{km}{day}$', labelpos='E',transform=ccrs.PlateCarree(),)

#storing data for mapping
all_speeds = []
angles_list = []
all_dates = pd.date_range(start='2024-1-1', end='2024-1-30', freq='D')

x_cr = (22, 33) # bottom, top
y_cr = (83, 92) #left, right

# all_speeds = np.array([])
all_speeds = []
angles_list = []
all_dates = pd.date_range(start='2024-1-1', end='2024-1-30', freq='D')

for date in all_dates:

    OSISAF = grab_OSISAF_drift(date = date, 
                          main_path = '/Volumes/Seagate2/OSI_SAF_Drift/')
    
#figure out the cropping

    crop_dx = OSISAF['dx'][y_cr[0]:y_cr[1], x_cr[0]:x_cr[1]]/2
    crop_dy = OSISAF['dy'][y_cr[0]:y_cr[1], x_cr[0]:x_cr[1]]/2
    region_speed = np.sqrt(np.nanmean(crop_dx)**2 + np.nanmean(crop_dy)**2)
    all_speeds.append(region_speed)
    
    #angle
    angle = math.atan((np.nanmean(crop_dx))/np.nanmean(crop_dy)
             )
    angles_list.append(math.degrees(angle))

print(len(all_speeds))
print(angles_list)

for date in all_dates:

    OSISAF = grab_OSISAF_drift(date = date, 
                          main_path = '/Volumes/Seagate2/OSI_SAF_Drift/')
    
#figure out the cropping

    crop_dx = OSISAF['dx'][y_cr[0]:y_cr[1], x_cr[0]:x_cr[1]]/2
    crop_dy = OSISAF['dy'][y_cr[0]:y_cr[1], x_cr[0]:x_cr[1]]/2
    region_speed = np.sqrt(np.nanmean(crop_dx)**2 + np.nanmean(crop_dy)**2)
    all_speeds.append(region_speed)
    
    #angle
    angle = math.atan((np.nanmean(crop_dx))/np.nanmean(crop_dy)
             )
    angles_list.append(math.degrees(angle))

### Atmospheric wind

In [ ]:

# specify date to import
date_list = pd.date_range(datetime(2024, 1, 1), datetime(2024, 1, 31)) #make this a date list
#============================

file_path = '/Volumes/Seagate2/Jan2024ERA5.nc'

ds = xr.load_dataset(file_path)
ds.close

ds_date = ds.sel(time = date)

ERA5 = {}

ERA5['time'] = ds_date.time.values
ERA5['lon'] = ds_date.longitude.values
ERA5['lat'] = ds_date.latitude.values

ERA5['lon_grid'], ERA5['lat_grid'] = np.meshgrid(ERA5['lon'], ERA5['lat'])

ERA5['u10'] = ds_date.u10.values
ERA5['v10'] = ds_date.v10.values
ERA5['t2m'] = ds_date.t2m.values
ERA5['msl'] = ds_date.msl.values

#create empty lists
u10_average = []
v10_average = []
t2m_average = []
date_axs = []

#averaging the values for a given date
for date in date_list:
    #subsetting data
    date_axs.append(date.to_pydatetime)
    ds_subset = ds.sel(longitude = slice(-153, -135), latitude = slice(73, 69), time = slice(date, date + timedelta(hours=23)))
    u10_average.append(ds_subset.mean(dim=("time", "latitude", "longitude")).u10.values)
    t2m_average.append((ds_subset.mean(dim=("time", "latitude", "longitude")).t2m.values)-273.15) #convert to Celsius
    v10_average.append((ds_subset.mean(dim=("time", "latitude", "longitude")).v10.values))


### Wind Factor time series

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(all_dates, angles_list)

#formatting the plot
plt.xticks(rotation=30)
plt.xticks(all_dates)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d'))
#labels
plt.xlabel('Date')
# ax.set_ylabel('sea ice speed (km/day)')

ax.set_ylabel('angle (degrees)')

plt.title(f'Sea ice drift angle {all_dates[0].strftime("%d-%b-%Y")} to {all_dates[-1].strftime("%d-%b-%Y")} over polynya')

### export code to a csv file

In [ ]:
d = {'time': all_dates, 'angle_deg': angles_list, 'u10': u10_average, ''}
df = pd.DataFrame(data=d)
df.to_csv('./Data/speeds_ice.csv', index=None) #removes Index column
df = pd.read_csv('./Data/speeds_ice.csv')
df['time'] = pd.to_datetime(df.time)